# Python Worksheet: Maximum Likelihood for a Nonlinear Problem

Dear students,

I am sorry for handing out the worksheet so late. My team and I tried hard to get a useful exercise for MLE. MLE is a crucial tool for all data science. In addition, our standard problem set was handed out as part of the python lecture notes. We hence had to come up with something useful, yet, also doable at the BSc level.

The following exercise is the result. We will likely spend some time discussing your and my work during the upcoming prof cafe.


If a significant number of learning groups communicate back that the task is too hard to accomplish till monday, I am willing to extend the final submission by one week. Yet, there will be (a then shorter) PS on vol modeling ARCH due in the same week.


## Remember this Crucial FACT

**Do NOT numerically optimize a Gaussian likelihood for a LINEAR model. Instead, take the ML parameter estimates from an OLS fit AND the ML estimate for the volatility from the respective residual**

$$
\sqrt{\frac{1}{T} \epsilon_{ols}' \epsilon_{ols}}
$$

##  A Useful Exercise had to be Non-Linear

Reason: see above

## G.0 Set-up of the Python Challenge

Have you ever wondered how central banks, banks, hedge funds and investors know whether a government bond is fairly priced, overpriced or undervalued? Well, they use the approach of "net present value" to figure out what the price of a bond should be. To do so, one has to get accurate forecasts of how the short rate and risk premia move into the future. A popular approach relies on so called "short rate models". These models fit the time series of the short rate to a realistic, yet convenient, parametrization. This allows to obtain a forecast of where short rates will be in the future. You need to do the same for the risk premium. You might wonder why do government bonds pay a risk premium. Well, the longer the maturity of the bond, the higher the interest rate sensitivity and hence, the higher (usually) the (maturity) premium. That premium compensates for the fact that future realized interest rates will be different from todays expected future rates. Based on the CAPM, you could say that interest rate risk is systematic and hence pays a risk premium. That is for the background.

A very popular short rate model is called "Vasicek model". That is because a mathematician, Dr. Vasicek, derived in the 1980s the term structure of interest rates under the assumption that the short rate follows a Gaussian distribution. It is fair to says that the assumed short rate follows a G-LRM. Based on stochastic calculus and the principle of net present value and no arbitrage, Dr. Vasicek derived the price of risk-free (non defaultable) bonds for different maturities. 

The Gaussian PDF of the short rate is

$$
r_t | F_{t-1} \sim \mathcal N\left(\theta^P (1- e^{-\kappa^P})+e^{-\kappa^P} r_{t-1}, \sigma^2 * \frac{1-e^{-2\kappa^P}}{2\kappa^P}\right).
$$

Note, $\theta^P$ is the long-run mean of $r$, $kappa^P$ is the speed of mean reversion and $\sigma$ is the instantaneous standard deviation of the interest rate shock. As the model of Vasicek is written in continuous time, the above PDF takes several exponentials and ratios, which are the result of an integration that is NOT subject to our BSc course.

Also, Dr. Vasicek derived that the arbitrage-free price of a bond at time $t$ with maturity in $\tau$ periods, $P_t(\tau)$ coincides with

$$
P_t(\tau) = e^{A(\tau) - B(\tau) \times r_t}
$$

with
\begin{align*}
B(\tau) &:= \frac{1-e^{-\kappa^Q \tau}}{\kappa^Q} \\
A(\tau) &:= (\theta^Q - \frac{\sigma^2}{2 (\kappa^Q)^2}) \times (  B(\tau) - \tau ) - \frac{\sigma^2}{4\kappa^Q} B^2(\tau) 
\end{align*}


Note, from the CAPM and NPV you know that prices of financial assets are discounted by the risk-free rate plus risk premium. The parameters $\theta^Q$ and $\kappa^Q$ differ from $\theta^P$ and $\kappa^P$, respectively. The former are adjusted by risk premiums. Intuitively, it is correct to think that the former are risk premium adjusted expressions of the latter. E.g. if the long-run mean of the short rate was at 2\%, it could be that bonds are priced as if the long-run rate was at 3\%. This means the price is lower than if the future payoff was discounted by 2\%. That extra reduction in price can be correctly interpreted as the result of a risk premium. That was to show you that the so called "Q parameters" are risk premium adjusted (and extracted from prices), while the $P$ parameters are extracted from empirical time-series data. I provide a formal and intuitive introduction into these asset pricing concepts in an upcoming MSc class on financial machine learning.

For now, you need only to keep in mind that the parameters of the model are $\kappa^P, \theta^P$ for the short rate PDF and $\theta^Q, \kappa^Q$ for the bond prices. The parameter $\sigma$ affects both, the short rate PDF and the bond price PDF.

Note, prices are non-stationary! Hence, we work with continuousy compounded yields
\begin{align*}
y_t(\tau) &:= - \frac{1}{\tau} \ln P_t(\tau)  
\end{align*}
 




## G.1 Data

You will work with the same interest rate data than a couple of weeks ago. The rates are in "GovBondYields.xls". 

Take the 12-, 60 and 120 months interest rates as your yields of interest, $y$. Take the 3-month yield as the short rate $r$.

**Units:** Divide the rates data by 1200 to arrive at monthly values in decimals. Work with these units.

 


## G.2 Financial Data Science Challenge

Your data science task is to extract the time-series of the expected market price of risk that is priced into bonds. You can consider the latter to be the priced-in Expected Sharpe Ratio in bonds (expected risk premium per unit of risk). The precise formula is given below. 


## G.3 Battle Plan

The battle plan is as follows. 

1. Write down the joint log likelihood function for $r_t$ and $y_t(\tau), \tau \in [12,60,120]$. For that assume a pairwise independent Gaussian measurement error for all yields. Further assume each of these measurement errors shares the same volatility, which we call
$$
\sigma_y.
$$
Note, $\sigma_y$ will be part of the parameter vector over which you optimize.

Note, you can interpret observed yields as Vasicek (model)-implied yield $y_t(\tau)$ plus a Gaussian measurement error that is drawn from $N(0,\sigma^2_y)$.

Note: the above assumption implies that the joint log likelihood function can be treated as the sum of 4 pairwise independent log likelihood functions. The independence refers to the respective residual being independent of the residuals of the other log likelihood. The parameters do of course affect all of the log likelihoods; although strictly speaking, $\kappa^P, \theta^P$ affect only the log likelihood for $r$, while $\kappa^Q, \theta^Q$ affect only the log likelhood for the 3 yields.

2. Write a python function that implements the joint log likelihood function. More recent research has shown you could indeed fit the PDF for $r$ independently for the PDF of $y$. Yet, here, we optimize the joint log likelihood.

3. Play a bit around with starting values for the optimization. That simple example reveals already, that you need to choose starting values carefully to end up in the global optimum. Regardless of your playing around, use at least the following procedure to find smart starting values

3.1 fit $r$ to an OLS and recover the ML estimates for $\kappa^P, \theta^P, \sigma$. Make these parameters be the respective starting values $\kappa^P, \theta^P, \sigma$ in  the joint log likelihood optimization. 

3.2 for the starting values of the Q parameters, we assume a 0 risk premium. Hence $\kappa^Q_{startValue} = \kappa^P_{startValue}$ and $\theta^Q_{startValue} = \theta^P_{startValue}$. Double check: inside the optimization, Q and P parameters can differ(!). P parameters affect $r$ while $Q$ parameters affect yields (!)

3.3 For the starting value of $\sigma_y$ we do the following: Regress (OLS) the 60month yield onto $r$ and take the volatility of the residual as the starting value for $\sigma_y$. 


4. The optimization routine is similar to the ipynb from class

4.1 use sco.minimize

4.2 use L-BFGS-B as the optimization routine

4.3 use a tol of 1e-8

4.5 minimize the negative joint log likelihood function

4.6 the upper bound for all parameters is 100. 

4.7 the lower bound for all parameters is -100, except for $\sigma$ and $\sigma_y$ who lower bound we set to 0.000001.

5. Estimate the model parameters of Vasicek.

6. Inside the vasicek model, the priced-in market price of risk (Sharpe ratio) in bonds is

$$
RP_t = \lambda_0 + \lambda_1 \times r_t
$$

with
\begin{align*}
\lambda_0 &:= \frac{\kappa^P \theta^P - \kappa^Q \theta^Q}{\sigma_r} \\
\lambda_1 &:= \frac{\kappa^Q - \kappa^P}{\sigma_r}.
\end{align*}

Here, you see that spread of Q and P parameters captures risk premium information. A formal explanation of that is not doable in the BSc course on Financial Data Science.

# Start of Exercise

## Data Preparation

In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import statsmodels.api as sm

In [11]:
df = pd.read_excel('GovBondYields.xls')

In [12]:
df = df.set_index('Date', drop=True)

In [13]:
df = df/1200

In [14]:
r = df[3]
y_12 = df[12]
y_60 = df[60]
y_120 = df[120]

## Regression to arrive at starting values for $\kappa^P$ and $\theta^P$ and sigma

In [205]:
endog = r[1:]
exog = r[:-1].values
exog = sm.add_constant(exog)
model = sm.OLS(endog=endog, exog=exog)

In [206]:
res = model.fit()

In [207]:
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                      3   R-squared:                       0.974
Model:                            OLS   Adj. R-squared:                  0.974
Method:                 Least Squares   F-statistic:                 2.339e+04
Date:                Fri, 11 Jun 2021   Prob (F-statistic):               0.00
Time:                        14:11:51   Log-Likelihood:                 4033.2
No. Observations:                 624   AIC:                            -8062.
Df Residuals:                     622   BIC:                            -8054.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        6.89e-05   3.17e-05      2.170      0.0

In [80]:
ols_kappa_P = - np.log(res.params[1])

In [81]:
ols_kappa_P

0.014895332457501307

In [82]:
ols_theta_P = res.params[0]/(1-res.params[1])

In [83]:
ols_theta_P

0.004660456741451862

In [106]:
ols_sigma = np.sqrt(res.mse_resid*2*ols_kappa_P/(1-np.exp(-2*ols_kappa_P)))

In [107]:
ols_sigma

0.00038075877416762224

In [138]:
ols_r_params = [ols_theta_P, ols_kappa_P, ols_sigma]

In [237]:
# Calculating est_sigma_y
endog = y_60.values
exog = r
exog = sm.add_constant(exog)
model = sm.OLS(endog=endog, exog=exog)

In [238]:
res = model.fit()

In [239]:
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.885
Model:                            OLS   Adj. R-squared:                  0.885
Method:                 Least Squares   F-statistic:                     4811.
Date:                Fri, 11 Jun 2021   Prob (F-statistic):          3.27e-295
Time:                        15:11:12   Log-Likelihood:                 3589.2
No. Observations:                 625   AIC:                            -7174.
Df Residuals:                     623   BIC:                            -7166.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0013   6.52e-05     20.597      0.0

In [240]:
est_sigma_y = np.sqrt(res.mse_resid)

In [241]:
est_sigma_y

0.0007770131789906451

## MLE

1. Write down the joint log likelihood function for $r_t$ and $y_t(\tau), \tau \in [12,60,120]. \sigma_y $ is assumed to be homoscedastic.

In [278]:
def B(tau, kappa_Q):
    return (1-np.exp(-kappa_Q*tau))/kappa_Q

def A(tau, theta_Q, kappa_Q, sigma):
    result =  (theta_Q - (sigma**2)/(2*kappa_Q**2))*(B(tau, kappa_Q) - tau) - (sigma**2)/(4*kappa_Q)*B(B(tau, kappa_Q), kappa_Q) # (??) B^2(tau) or B(tau)^2
    return result
    
    
def vasicek_bond_price(tau, theta_Q, kappa_Q, sigma, r_t):
    P_t = np.exp(A(tau, theta_Q, kappa_Q, sigma) - B(tau, kappa_Q)*r_t)
    return P_t

def vasicek_bond_yield(tau, theta_Q, kappa_Q, sigma, r_t):
    P_t = vasicek_bond_price(tau, theta_Q, kappa_Q, sigma, r_t)
    vbq = -np.log(P_t)/tau
    return vbq

def r_log_likelihood(theta_P, kappa_P, sigma, sigma_y):
    endog = r[1:]
    exog = r[:-1]
    log_likelihood = 0
    for r_tm1, r_t in zip(exog, endog):
        distr = stats.norm(theta_P*(1-np.exp(-kappa_P))+np.exp(-kappa_P)*r_tm1, sigma_y + np.sqrt(sigma**2*(1-np.exp(-2*kappa_P))/(2*kappa_P)))
        log_likelihood_t = np.log(distr.pdf(r_t))
        log_likelihood += log_likelihood_t
    return log_likelihood

def neg_log_likelihood(params):
    print(params)
    # Q's are for y P's are for r 
    theta_Q, kappa_Q, theta_P, kappa_P, sigma, sigma_y = params
    
    # yield likelihood
    wnp = stats.norm(0, sigma_y)
    vby_12 = r.apply(lambda x: vasicek_bond_yield(12, theta_Q, kappa_Q, sigma, x))
    vby_60 = r.apply(lambda x: vasicek_bond_yield(60, theta_Q, kappa_Q, sigma, x))
    vby_120 = r.apply(lambda x: vasicek_bond_yield(120, theta_Q, kappa_Q, sigma, x))
    vby_log_likelihood = sum([np.log(wnp.pdf(np.abs(x - z))).sum() for x, z in [(y_12, vby_12), (y_60, vby_60), (y_120, vby_60)]])
    #print(f"vby_log_likelihood: {vby_log_likelihood}")
    
    # short rate likelihood
    short_rate_log_likelihood = r_log_likelihood(theta_P, kappa_P, sigma, sigma_y)
    #print(f"short_rate_log_likelihood: {short_rate_log_likelihood}")
    
    overall_log_likelihood = vby_log_likelihood + short_rate_log_likelihood
    print(f"overall_log_likelihood: {overall_log_likelihood}")
    return -overall_log_likelihood

In [279]:
ols_r_params

[0.004660456741451862, 0.014895332457501307, 0.00038075877416762224]

In [280]:
A(12, 10, -10, ols_sigma) == np.inf

<ipython-input-278-34ed90086cff>:2: RuntimeWarning: overflow encountered in exp
  return (1-np.exp(-kappa_Q*tau))/kappa_Q


True

In [281]:
wnp = stats.norm(0, est_sigma_y)
z = r.apply(lambda x: vasicek_bond_price(12, 100, -100, 0.00038075877416762224, x))
z
#wnp.pdf(np.abs(r-z))

Date
1954-04-01   NaN
1954-05-01   NaN
1954-06-01   NaN
1954-07-01   NaN
1954-08-01   NaN
              ..
2005-12-01   NaN
2006-01-01   NaN
2006-02-01   NaN
2006-03-01   NaN
2006-04-01   NaN
Name: 3, Length: 625, dtype: float64

In [255]:
init_params = np.array([100, -100, ols_theta_P, ols_kappa_P, ols_sigma, est_sigma_y])

In [256]:
neg_log_likelihood(init_params)

[ 1.00000000e+02 -1.00000000e+02  4.66045674e-03  1.48953325e-02
  3.80758774e-04  7.77013179e-04]
vby_log_likelihood: nan
overall_log_likelihood: nan


nan

2. Write a python function that implements the joint log likelihood function. More recent research has shown you could indeed fit the PDF for $r$ independently for the PDF of $y$. Yet, here, we optimize the joint log likelihood.

In [318]:
# Make bounds for parameters:
bounds = ((-0.5, 0.5),
         (-2, 5),
         (-2, 5),
         (-2, 5),
         (0.000001, 10),
         (0.000001, 10))

In [319]:
import scipy.optimize as sco

In [320]:
init_params = np.array([ols_theta_P, 
                        ols_kappa_P, 
                        ols_theta_P, 
                        ols_kappa_P, 
                        ols_sigma, 
                        est_sigma_y])

In [321]:
init_params

array([0.00466046, 0.01489533, 0.00466046, 0.01489533, 0.00038076,
       0.00077701])

In [325]:
MLE = sco.minimize(fun = neg_log_likelihood, 
                   x0 = init_params*1.05, 
                   bounds=bounds, 
                   method='L-BFGS-B', 
                   tol=1e-8, 
                   options={'disp':True})

[0.00489348 0.0156401  0.00489348 0.0156401  0.0003998  0.00081586]
overall_log_likelihood: 13070.90597717192
[0.00489349 0.0156401  0.00489348 0.0156401  0.0003998  0.00081586]
overall_log_likelihood: 13070.91289674141
[0.00489348 0.01564011 0.00489348 0.0156401  0.0003998  0.00081586]
overall_log_likelihood: 13070.905891999782
[0.00489348 0.0156401  0.00489349 0.0156401  0.0003998  0.00081586]
overall_log_likelihood: 13070.905976917915
[0.00489348 0.0156401  0.00489348 0.01564011 0.0003998  0.00081586]
overall_log_likelihood: 13070.905978063029
[0.00489348 0.0156401  0.00489348 0.0156401  0.00039981 0.00081586]
overall_log_likelihood: 13070.890217539294
[0.00489348 0.0156401  0.00489348 0.0156401  0.0003998  0.00081587]
overall_log_likelihood: 13070.93046935586
[ 5.e-01 -2.e+00 -2.e+00  5.e+00  1.e-06  1.e+01]


<ipython-input-278-34ed90086cff>:38: RuntimeWarning: divide by zero encountered in log
  vby_log_likelihood = sum([np.log(wnp.pdf(np.abs(x - z))).sum() for x, z in [(y_12, vby_12), (y_60, vby_60), (y_120, vby_60)]])


overall_log_likelihood: -inf
[ 4.9999999e-01 -2.0000000e+00 -2.0000000e+00  5.0000000e+00
  1.0000000e-06  1.0000000e+01]
overall_log_likelihood: -inf
[ 5.00000000e-01 -1.99999999e+00 -2.00000000e+00  5.00000000e+00
  1.00000000e-06  1.00000000e+01]
overall_log_likelihood: -inf
[ 5.00000000e-01 -2.00000000e+00 -1.99999999e+00  5.00000000e+00
  1.00000000e-06  1.00000000e+01]
overall_log_likelihood: -inf
[ 5.00000000e-01 -2.00000000e+00 -2.00000000e+00  4.99999999e+00
  1.00000000e-06  1.00000000e+01]
overall_log_likelihood: -inf
[ 5.00e-01 -2.00e+00 -2.00e+00  5.00e+00  1.01e-06  1.00e+01]
overall_log_likelihood: -inf
[ 5.00000000e-01 -2.00000000e+00 -2.00000000e+00  5.00000000e+00
  1.00000000e-06  9.99999999e+00]
overall_log_likelihood: -inf
[0.00489348 0.0156401  0.00489348 0.0156401  0.0003998  0.00081586]
overall_log_likelihood: 13070.90597717192
[0.00489349 0.0156401  0.00489348 0.0156401  0.0003998  0.00081586]
overall_log_likelihood: 13070.91289674141
[0.00489348 0.01564011 0.0

In [326]:
MLE

      fun: -13070.90597717192
 hess_inv: <6x6 LbfgsInvHessProduct with dtype=float64>
      jac: array([-6.91956949e+05,  8.51721379e+03,  2.54005499e+01, -8.91108357e+01,
        1.57596326e+06, -2.44921839e+06])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 21
      nit: 1
     njev: 3
   status: 0
  success: True
        x: array([0.00489348, 0.0156401 , 0.00489348, 0.0156401 , 0.0003998 ,
       0.00081586])

3. Play a bit around with starting values for the optimization. That simple example reveals already, that you need to choose starting values carefully to end up in the global optimum. Regardless of your playing around, use at least the following procedure to find smart starting values